In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
path="D:/Data_analytics/project/NYC_Taxi_Trip/NYC_Taxi_Trip/train.csv"

In [ ]:
NYC_df= pd.read_csv(path,parse_dates=True)

In [ ]:
NYC_df.head(5)

In [ ]:
NYC_df.info()

In [ ]:
NYC_df.isnull().sum()

In [ ]:
import datetime as dt 
import pytz

In [ ]:
NYC_df.hist('trip_duration',bins=10)

In [ ]:
NYC_df['pickup_datetime'] = pd.to_datetime(NYC_df['pickup_datetime'])
NYC_df['dropoff_datetime']= pd.to_datetime(NYC_df['dropoff_datetime'])

In [ ]:
NYC_df['pickup_date'] = pd.to_datetime(NYC_df['pickup_datetime']).dt.date

In [ ]:
NYC_df['pickup_date']

In [ ]:
NYC_df['hour'] = pd.to_datetime(NYC_df['pickup_datetime']).dt.hour
#sales['time_hour'] = pd.to_datetime(sales['timestamp']).dt.hour
#NYC_df['hour']

In [ ]:
NYC_df.hist('hour',bins=4)

In [ ]:
NYC_df['pickup_time']

In [ ]:
NYC_df['pickup_date'] = pd.to_datetime(NYC_df['pickup_date'], format = '%Y-%m-%d')
NYC_df['weekday'] = NYC_df['pickup_date'].dt.dayofweek

In [ ]:
days = {0:'Mon',1:'Tues',2:'Weds',3:'Thurs',4:'Fri',5:'Sat',6:'Sun'}
NYC_df['weekday'] = NYC_df['weekday'].apply(lambda x: days[x])

In [ ]:
NYC_df['weekday']

In [ ]:
percentiles = NYC_df['trip_duration'].quantile([0.01,0.99]).values
NYC_df['trip_duration'][NYC_df['trip_duration'] <= percentiles[0]] = percentiles[0]
NYC_df['trip_duration'][NYC_df['trip_duration'] >= percentiles[1]] = percentiles[1]

In [ ]:
import seaborn as sns
trip_duration_weekday_boxplot=sns.boxplot(x="weekday", y="trip_duration", data=NYC_df)

In [ ]:
trip_duration_weekday_barplot=sns.barplot(x="weekday", y="trip_duration", data=NYC_df)

In [ ]:
NYC_df['month'] = NYC_df['pickup_date'].dt.month
NYC_df['month']

In [ ]:
import matplotlib.pyplot as plt
NYC_df.hist('month',bins=6)

In [ ]:
days = {1:'Jan',2:'Feb',3:'Mar',4:'April',5:'May',6:'June'}
NYC_df['month'] = NYC_df['month'].apply(lambda x: days[x])

In [ ]:
import seaborn as sns
trip_duration_weekday_barplot=sns.barplot(x="month", y="trip_duration", data=NYC_df)

In [ ]:
labels = ['Morning', 'Evening', 'Night']
NYC_df['Day'] = pd.cut(NYC_df.hour, bins=3, labels=labels)

In [ ]:
trip_duration_weekday_barplot=sns.barplot(x="Day", y="trip_duration", data=NYC_df)

In [ ]:
grid = sns.FacetGrid(NYC_df, col='weekday', row='Day', height=2.2, aspect=1.6)
grid.map(sns.barplot, 'trip_duration')
grid.add_legend();

In [ ]:
trip_duration_vendorid_boxplot=sns.boxplot(x="vendor_id", y="trip_duration", data=NYC_df)

In [ ]:
NYC_df['log_trip_duration'] = np.log(NYC_df['trip_duration'].values + 1)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(NYC_df['log_trip_duration'].values, bins=100)
plt.xlabel('log(trip_duration)')
plt.ylabel('number of train records')

In [ ]:
def haversine_array(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

In [ ]:
pickup_longitude=NYC_df['pickup_longitude']
pickup_latitude=NYC_df['pickup_latitude']
dropoff_longitude=NYC_df['dropoff_longitude']
dropoff_latitude=NYC_df['dropoff_latitude']

NYC_df['distance']=haversine_array(pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude)

In [ ]:
NYC_df['distance']=NYC_df['distance']*1000

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(NYC_df['distance'], NYC_df['log_trip_duration'], alpha=0.5)

In [ ]:
trip_duration_weekday_barplot=sns.barplot(x="weekday", y="log_trip_duration", data=NYC_df)

In [ ]:
NYC_df.info()

In [ ]:
weekday_dummies = pd.get_dummies(NYC_df.weekday, prefix='weekday')
month_dummies = pd.get_dummies(NYC_df.month, prefix='month')
Day_dummies = pd.get_dummies(NYC_df.Day, prefix='Day')
Vendor_dummies = pd.get_dummies(NYC_df.vendor_id, prefix='vendor_id')

In [ ]:
weekday_dummies.drop(weekday_dummies.columns[0], axis=1, inplace=True)
month_dummies.drop(month_dummies.columns[0], axis=1, inplace=True)
Day_dummies.drop(Day_dummies.columns[0], axis=1, inplace=True)
Vendor_dummies.drop(Vendor_dummies.columns[0], axis=1, inplace=True)

In [ ]:
NYC_df=pd.concat([NYC_df, weekday_dummies,month_dummies,Day_dummies,Vendor_dummies], axis=1)

In [ ]:
 NYC_df['distance'] = NYC_df['distance'].apply(np.int64)


In [ ]:
cols = ['weekday_Mon', 'weekday_Sat', 'weekday_Sun', 'weekday_Tues','weekday_Weds','month_Feb','month_Jan','month_June','month_Mar','month_May','Day_Evening','Day_Night','vendor_id_2']
for col in cols:
   NYC_df[col] = NYC_df[col].apply(lambda x: int(x) if x == x else "")

In [ ]:
NYC_df.info()

In [ ]:
NYC_df=NYC_df.drop(columns=['id','dropoff_datetime','pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','id','pickup_date','hour','weekday','month','Day'])


In [ ]:
NYC_df=NYC_df.drop(columns=['store_and_fwd_flag'])

In [ ]:
Y=NYC_df["log_trip_duration"]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X=NYC_df.drop(columns=['trip_duration','log_trip_duration','store_and_fwd_flag'], axis=1)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=123)

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

In [ ]:
linreg.fit(X_train, y_train)

In [ ]:
y_pred = linreg.predict(X_test)

In [ ]:
from sklearn import metrics
print ('MAE:', metrics.mean_absolute_error(y_test, y_pred))
print ('MSE:', metrics.mean_squared_error(y_test, y_pred))
print ('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
y_null = np.zeros_like(y_test, dtype=float)
y_null.fill(y_test.mean())
np.sqrt(metrics.mean_squared_error(y_test, y_null))

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cv_scores = cross_val_score(linreg, X, Y, cv=5)

In [ ]:
print(cv_scores)

In [ ]:
print("Average 5-Fold CV Score: {}".format(np.mean(cv_scores)))

In [ ]:
from sklearn.linear_model import Ridge
alpha_space = np.logspace(-4, 0, 50)
ridge_scores = []
ridge_scores_std = []
ridge = Ridge(normalize=True)

for alpha in alpha_space:

    # Specify the alpha value to use: ridge.alpha
    ridge.alpha = alpha
    
    # Perform 10-fold CV: ridge_cv_scores
    ridge_cv_scores = cross_val_score(ridge, X, Y, cv=10)
    
    # Append the mean of ridge_cv_scores to ridge_scores
    ridge_scores.append(np.mean(ridge_cv_scores))
    
    # Append the std of ridge_cv_scores to ridge_scores_std
    ridge_scores_std.append(np.std(ridge_cv_scores))
    

In [ ]:
print("Average 5-Fold CV Score: {}".format(np.mean(ridge_cv_scores)))

In [ ]:
def display_plot(cv_scores, cv_scores_std):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(alpha_space, cv_scores)

    std_error = cv_scores_std / np.sqrt(10)

    ax.fill_between(alpha_space, cv_scores + std_error, cv_scores - std_error, alpha=0.2)
    ax.set_ylabel('CV Score +/- Std Error')
    ax.set_xlabel('Alpha')
    ax.axhline(np.max(cv_scores), linestyle='--', color='.5')
    ax.set_xlim([alpha_space[0], alpha_space[-1]])
    ax.set_xscale('log')
    plt.show()

# Display the plot
display_plot(ridge_scores, ridge_scores_std)